In [1]:
import evaluate
import process
text = 'data/brown.txt'

In [2]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import casual

import json

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import numpy as np
import argparse
import numpy as np
from scipy.stats import spearmanr

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cosine
from numpy.linalg import norm

import random
import os

from process import load_model, load_msr

In [3]:
windowList = [2,5,10]
dimList = [100,300,1000]
EPOCHS = 5
iwin = 2
idim = 100

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
len(stop_words)

179

In [5]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        for line in open(self.filename, 'r'):
            yield [word.lower() for word in tokenizer.tokenize(line) if word.lower() not in stop_words]
sentence = MySentences(text)

In [6]:
# count unigrams
from collections import Counter
word_counts = Counter()
for x in sentence:
    word_counts.update(x)

In [7]:
print(len(word_counts))
print(word_counts.most_common(5))
print(word_counts.most_common()[-5:])

42281
[('one', 3504), ('would', 2719), ('said', 1961), ('time', 1695), ('new', 1646)]
[('perelman', 1), ('exhaling', 1), ('aviary', 1), ('boucle', 1), ('stupefying', 1)]


In [55]:
# minimum word count = 5
word_counts2 = Counter()
for key,value in word_counts.items():
    if value >= 5:
        word_counts2[key] = value
print(len(word_counts2))
print(word_counts2.most_common(5))
print(word_counts2.most_common()[-5:])

13891
[('one', 3504), ('would', 2719), ('said', 1961), ('time', 1695), ('new', 1646)]
[('furrow', 5), ('richert', 5), ('kafka', 5), ('poitrine', 5), ('quasimodo', 5)]


In [8]:
# count co-occurance
joint = Counter()
for isen in sentence:
    for j, word in enumerate(isen):
        if word not in word_counts:
            continue
        index_min = max(0, j-iwin)
        index_max = min(len(isen), j+iwin+1)
        index = [ii for ii in range(index_min, index_max) if ii!=j]
        for iin in index:
            joint[(word, isen[iin])] += 1
print(len(joint))
print(joint.most_common(5))

1497145
[(('united', 'states'), 401), (('states', 'united'), 401), (('af', 'af'), 392), (('new', 'york'), 312), (('york', 'new'), 312)]


In [82]:
word_counts2['new']

1646

In [9]:
# generate index for words in vocabulary
index_dict = dict()
word_dict = dict()
ii = 0
for i in word_counts:
    index_dict[i] = ii
    word_dict[ii] = i
    ii += 1
    
# transform count joint to sparse matrix
import math
row_index = []
col_index = []
values = []
for (wi,wj), count in joint.items():
    row = index_dict[wi]
    col = index_dict[wj]
    prod = word_counts[wi]*word_counts[wj]*len(joint)/(len(word_counts)**2)
    if count > prod:
        value = math.log(count/prod)
    else:
        value = 0
    row_index.append(row)
    col_index.append(col)
    values.append(value)

from scipy import sparse
ppmi = sparse.csr_matrix((values, (row_index, col_index)))

In [10]:
ppmi.nonzero()

(array([    0,     0,     0, ..., 42279, 42280, 42280], dtype=int32),
 array([    1,     2,     3, ..., 42280,  6523, 42279], dtype=int32))

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
def ww_sim(word, mat, topn=10):
    indx = index_dict[word]
    if isinstance(mat, sparse.csr_matrix):
        v1 = mat.getrow(indx)
    else:
        v1 = mat[indx:indx+1, :]
    sims = cosine_similarity(mat, v1).flatten()
    sindxs = np.argsort(-sims)
#     print(sindxs[0:topn])
#     print(index_dict[9254])
#     print(sims[9254])
    sim_word_scores = [(word_dict[sindx], sims[sindx]) for sindx in sindxs[0:topn]]
    return sim_word_scores

In [19]:
ww_sim('uncle', ppmi)

[('uncle', 1.0),
 ('replanted', 0.19015442139538086),
 ('ffortescue', 0.18770341268912027),
 ('winnings', 0.1635886263178493),
 ('lorde', 0.14375627841760283),
 ('countrey', 0.14093206890510743),
 ('tussle', 0.13681876061169834),
 ('donald', 0.12825493146537434),
 ('underlay', 0.12695794690543205),
 ('stowe', 0.12417572821154328)]

In [15]:
from scipy.sparse import linalg
uu,ss,vv = linalg.svds(ppmi, idim)
print(np.shape(uu),np.shape(ss),np.shape(vv))

(42281, 100) (100,) (100, 42281)


In [16]:
unorm = uu / np.sqrt(np.sum(uu*uu, axis=1, keepdims=True))
vnorm = vv / np.sqrt(np.sum(vv*vv, axis=0, keepdims=True))
#word_vecs = unorm
#word_vecs = vnorm.T
word_vecs = uu + vv.T
word_vecs_norm = word_vecs / np.sqrt(np.sum(word_vecs*word_vecs, axis=1, keepdims=True))

/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [17]:
ww_sim("female",word_vecs)

[('female', 1.0000000000000004),
 ('choosy', 0.9837187516554718),
 ('parasite', 0.9767523687490764),
 ('matriarchal', 0.9579497486448949),
 ('amazons', 0.8838188316210213),
 ('dissenters', 0.8662304672042447),
 ('criticizing', 0.8620292444505535),
 ('colombian', 0.8430180909941202),
 ('merciless', 0.8284281431464853),
 ('factions', 0.8258929852339392)]

In [18]:
ww_sim("strike",word_vecs)

[('strike', 0.9999999999999998),
 ('soldiers', 0.804110429821317),
 ('unenthusiastic', 0.7801367386647553),
 ('gesturing', 0.7547880528517577),
 ('upraised', 0.7517724899072793),
 ('pressed', 0.7488827586046398),
 ('spruced', 0.7431111912300254),
 ('drifted', 0.7415945161082629),
 ('warmish', 0.7354715342614404),
 ('foul', 0.7351051738108774)]

In [20]:
ww_sim("uncle",word_vecs)

[('uncle', 1.0000000000000002),
 ('indisposition', 0.9978157634359457),
 ('doin', 0.9697134515392729),
 ('countrey', 0.9446329677743157),
 ('vinnicum', 0.943740872841573),
 ('conspires', 0.939735520007934),
 ('indisposed', 0.9332111812406453),
 ('ffortescue', 0.9292737667590284),
 ('exchequer', 0.9240052795821159),
 ('hush', 0.9179795742322708)]

In [42]:
text = "Sentences iterable can be simply a list, but for larger corpora, consider a generator that streams the sentences directly from disk/network"

In [86]:
print(ppmi[0,:])

  (0, 1)	0.0
  (0, 2)	0.0
  (0, 3)	0.0
  (0, 7)	0.0
  (0, 29)	0.0
  (0, 30)	0.0
  (0, 31)	0.0
  (0, 58)	0.0
  (0, 59)	0.0
  (0, 60)	0.0
  (0, 64)	0.0
  (0, 66)	0.0
  (0, 67)	0.0
  (0, 90)	0.0
  (0, 106)	0.0
  (0, 119)	0.0
  (0, 129)	0.0
  (0, 130)	0.0
  (0, 131)	0.0
  (0, 138)	0.0
  (0, 139)	0.0
  (0, 149)	0.0
  (0, 150)	0.0
  (0, 151)	0.0
  (0, 156)	0.0
  (0, 157)	0.0
  (0, 158)	0.0
  (0, 210)	0.0
  (0, 211)	0.0
  (0, 215)	0.0
  (0, 222)	0.0
  (0, 235)	0.0
  (0, 236)	0.0
  (0, 237)	0.0
  (0, 241)	0.6394438532335271
  (0, 242)	0.0
  (0, 243)	0.0
  (0, 244)	0.0
  (0, 246)	0.6394438532335271
  (0, 252)	0.0
  (0, 304)	0.0
  (0, 1051)	0.0
  (0, 1212)	0.0
  (0, 2258)	0.0
  (0, 4037)	0.0
  (0, 4165)	0.0
  (0, 6212)	0.0
  (0, 6789)	0.0


In [123]:
word_vecs.save("../../NLPdata/hw3/savedModel/test")

AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [124]:
np.shape(word_vecs)

(13891, 100)

In [125]:
word_vecs[0]

array([ 1.63849412e-04, -9.70924308e-06, -3.89196601e-04, -1.99606045e-04,
       -1.52572737e-04,  3.92952168e-04,  4.11227720e-16,  6.34080014e-04,
       -8.64653808e-05, -1.56360244e-03, -5.49293412e-15, -2.56142763e-17,
        1.16559588e-04, -4.07202049e-05,  4.32493179e-05,  2.21068619e-04,
        3.71345924e-04,  4.73482216e-04,  8.14642407e-17, -4.92336396e-04,
       -1.26622883e-04,  1.42008801e-15,  6.22602646e-04,  9.23833035e-05,
       -1.18927200e-04,  9.85675300e-17,  2.16759825e-05, -4.91414635e-16,
       -3.35825277e-04, -3.18863859e-16, -1.83852395e-04, -8.15787470e-05,
        2.11504127e-17, -8.56977114e-17, -6.59427894e-05,  2.85409917e-04,
       -1.09294355e-16,  6.32557204e-05,  5.77771367e-05,  8.97643609e-05,
       -4.61098228e-05,  2.48408268e-05, -3.19926818e-04, -9.76574519e-05,
       -5.61873316e-05,  6.93308960e-04,  3.10089953e-15, -1.51924772e-04,
       -2.84483907e-05,  1.10950504e-04, -5.57999895e-17,  1.62489884e-05,
       -1.04200530e-04, -